In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
sample_sub_data = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

In [ ]:
train_data.shape

In [ ]:
sample_sub_data.shape

In [ ]:
test_data

In [ ]:
test_data.head()

In [ ]:
x = train_data.iloc[:, 1:-1]
y = train_data['target']
x_test = test_data.iloc[:, 1:]
y_test = sample_sub_data.iloc[:, 1]

In [ ]:
x_train = pd.get_dummies(x)
y_train = pd.get_dummies(y)
x_test = pd.get_dummies(x_test)
y_test = pd.get_dummies(y_test)

In [ ]:
y_test

In [ ]:
missing_val = x.isnull().sum()
print(missing_val[missing_val > 0])

from sklearn.model_selection import train_test_split 

x, x_val, y, y_val = train_test_split(x, y, train_size = 0.85, random_state = 0)

In [ ]:
import keras 
from tensorflow.keras import layers 
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model = keras.Sequential([ 
    layers.BatchNormalization(),
    layers.Dense(2048, activation='sigmoid', input_shape=[101]),
    layers.Dropout(rate = 0.3), 
    layers.BatchNormalization(),    
    layers.Dense(2048, activation='sigmoid'), 
    layers.Dropout(rate = 0.3), 
    layers.BatchNormalization(),
    layers.Dense(2048, activation='sigmoid'), 
    layers.Dropout(rate = 0.3), 
    layers.Dense(1, activation = 'sigmoid'), 
]) 

early_stopping = EarlyStopping(min_delta = 0.001, patience = 10, restore_best_weights = True)
reducedlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

In [ ]:
model.compile( 
    optimizer='adamax', 
    loss='binary_crossentropy', 
    metrics=['AUC'],
    )

In [ ]:
prediction = model.fit( 
    x, y, 
    validation_data=(x_val, y_val), 
    batch_size=256, 
    epochs=100, 
    callbacks = [
                early_stopping,reducedlr], 
    verbose = 2, 
)

In [ ]:
preds_test = model.predict(x_test)

In [ ]:
sample_sub_data['target'] = np.mean(preds_test, axis = 1) 
sample_sub_data.to_csv('submission.csv', index=False)